<a href="https://colab.research.google.com/github/SushovitNanda/SemEval-Food-Hazards/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load the dataset
data = pd.read_csv('incidents_labelled.csv')

# Combine 'title' and 'text' columns for richer input data
data['combined_text'] = data['title'] + " " + data['text']

# Encode hazard-category to numerical labels
label_mapping = {label: idx for idx, label in enumerate(data['hazard-category'].unique())}
data['label'] = data['hazard-category'].map(label_mapping)

# Stratified train-test split
train_df, val_df = train_test_split(data, test_size=0.2, stratify=data['label'], random_state=42)

# Define tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_mapping))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# Custom Dataset Class for PyTorch
class HazardDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

# Create DataLoaders
train_dataset = HazardDataset(train_df['combined_text'].tolist(), train_df['label'].tolist(), tokenizer)
val_dataset = HazardDataset(val_df['combined_text'].tolist(), val_df['label'].tolist(), tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# Optimizer and Scheduler
optimizer = AdamW(model.parameters(), lr=1e-5, correct_bias=False)
total_steps = len(train_loader) * 5  # Assuming 5 epochs (was *3 initially for 3 epochs)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)


In [9]:
# Training function
def train_model(model, train_loader, val_loader, optimizer, scheduler, epochs=5):
    model.train()
    for epoch in range(epochs):
        total_train_loss = 0
        for batch in tqdm(train_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            model.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_train_loss += loss.item()
            loss.backward()
            optimizer.step()
            scheduler.step()

        avg_train_loss = total_train_loss / len(train_loader)
        print(f"Epoch {epoch + 1}/{epochs} - Training Loss: {avg_train_loss}")

        # Validation after each epoch
        evaluate_model(model, val_loader)

# Evaluation function
def evaluate_model(model, val_loader):
    model.eval()
    predictions, true_labels = [], []

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).flatten()

            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    print("Validation Classification Report:")
    print(classification_report(true_labels, predictions, target_names=label_mapping.keys()))


In [10]:
# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Train and evaluate the model
train_model(model, train_loader, val_loader, optimizer, scheduler)

100%|██████████| 300/300 [02:20<00:00,  2.13it/s]


Epoch 1/5 - Training Loss: 0.2622132652439177
Validation Classification Report:
                                precision    recall  f1-score   support

                    biological       0.94      0.95      0.95       404
                foreign bodies       0.91      0.84      0.87       154
                      chemical       0.82      0.90      0.86       100
                         fraud       0.77      0.62      0.69        82
          organoleptic aspects       0.69      0.69      0.69        13
                     allergens       0.89      0.97      0.93       391
              packaging defect       0.64      0.44      0.52        16
                  other hazard       0.67      0.48      0.56        29
food additives and flavourings       0.50      0.20      0.29         5
                     migration       0.00      0.00      0.00         3

                      accuracy                           0.89      1197
                     macro avg       0.68      0.61   

100%|██████████| 300/300 [02:18<00:00,  2.17it/s]


Epoch 2/5 - Training Loss: 0.15425923176730671
Validation Classification Report:
                                precision    recall  f1-score   support

                    biological       0.95      0.95      0.95       404
                foreign bodies       0.88      0.85      0.86       154
                      chemical       0.82      0.87      0.84       100
                         fraud       0.74      0.65      0.69        82
          organoleptic aspects       1.00      0.54      0.70        13
                     allergens       0.91      0.96      0.93       391
              packaging defect       0.57      0.50      0.53        16
                  other hazard       0.63      0.59      0.61        29
food additives and flavourings       0.67      0.40      0.50         5
                     migration       0.00      0.00      0.00         3

                      accuracy                           0.89      1197
                     macro avg       0.72      0.63  

100%|██████████| 300/300 [02:18<00:00,  2.16it/s]


Epoch 3/5 - Training Loss: 0.08533938772976399
Validation Classification Report:
                                precision    recall  f1-score   support

                    biological       0.94      0.97      0.96       404
                foreign bodies       0.92      0.86      0.89       154
                      chemical       0.86      0.88      0.87       100
                         fraud       0.78      0.66      0.72        82
          organoleptic aspects       0.89      0.62      0.73        13
                     allergens       0.91      0.96      0.93       391
              packaging defect       0.64      0.44      0.52        16
                  other hazard       0.58      0.62      0.60        29
food additives and flavourings       0.50      0.40      0.44         5
                     migration       0.67      0.67      0.67         3

                      accuracy                           0.90      1197
                     macro avg       0.77      0.71  

100%|██████████| 300/300 [02:17<00:00,  2.17it/s]


Epoch 4/5 - Training Loss: 0.05048398966745784
Validation Classification Report:
                                precision    recall  f1-score   support

                    biological       0.96      0.95      0.95       404
                foreign bodies       0.90      0.87      0.88       154
                      chemical       0.85      0.89      0.87       100
                         fraud       0.77      0.70      0.73        82
          organoleptic aspects       0.89      0.62      0.73        13
                     allergens       0.92      0.96      0.94       391
              packaging defect       0.62      0.50      0.55        16
                  other hazard       0.58      0.62      0.60        29
food additives and flavourings       0.50      0.40      0.44         5
                     migration       0.67      0.67      0.67         3

                      accuracy                           0.90      1197
                     macro avg       0.76      0.72  

100%|██████████| 300/300 [02:18<00:00,  2.17it/s]


Epoch 5/5 - Training Loss: 0.03448265280108899
Validation Classification Report:
                                precision    recall  f1-score   support

                    biological       0.96      0.95      0.96       404
                foreign bodies       0.89      0.88      0.89       154
                      chemical       0.86      0.89      0.87       100
                         fraud       0.78      0.70      0.74        82
          organoleptic aspects       0.89      0.62      0.73        13
                     allergens       0.92      0.96      0.94       391
              packaging defect       0.67      0.50      0.57        16
                  other hazard       0.59      0.66      0.62        29
food additives and flavourings       0.50      0.40      0.44         5
                     migration       0.67      0.67      0.67         3

                      accuracy                           0.90      1197
                     macro avg       0.77      0.72  